In [29]:
# Let's load and process the dataset
from fuel.datasets.dogs_vs_cats import DogsVsCats
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers.image import MinimumImageDimensions
from fuel.transformers import Flatten
from fuel.transformers import ScaleAndShift
from fuel.transformers import Cast

train_set = DogsVsCats(('train',), subset=slice(0, 20000))
valid_set = DogsVsCats(('train',), subset=slice(20000, 25000))
test_set = DogsVsCats(('test',))

batch_size = 16
n_train_batches = train_set.num_examples // batch_size

#################################################################################
# Train Stream
# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream instead of DataStream.default_stream constructor we return
# our images exactly as is.
stream = DataStream(
    train_set,
    iteration_scheme=ShuffledScheme(train_set.num_examples, batch_size)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to upscale images to at least (512 x 512)

upscale_stream = MinimumImageDimensions(stream, (128,128),which_sources=('image_features',))

# Take random crops of (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    upscale_stream, (32, 32), which_sources=('image_features',))

# Convert images to [0,1] scale
default_cropped_stream = ScaleAndShift(cropped_stream,  1.0/(255.0), 0., which_sources=('image_features',))
# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
train_stream = Flatten(
    default_cropped_stream, which_sources=('image_features',))
    
cast_train_stream = Cast(train_stream, dtype='float32', which_sources=('image_features',))

from fuel.server import start_server




In [4]:
import timeit




In [26]:
start_time = timeit.default_timer()
for x,y in stream.get_epoch_iterator():
    print(timeit.default_timer()-start_time)
    start_time = timeit.default_timer()

end_time = timeit.default_timer() - start_time

0.06365509901661426
0.05239932704716921
0.06027437397278845
0.052838695934042335
0.049352340050973
0.04439308203291148
0.04978604998905212
0.044726491905748844
0.050183604937046766
0.04957210610155016
0.045425528078339994
0.051342691062018275
0.04968431405723095
0.042780768941156566
0.045309294015169144
0.05423232400789857
0.04687606601510197
0.04649312503170222
0.04653058806434274
0.05214408994652331
0.046322031994350255
0.04814936697948724
0.04488540207967162
0.0481210770085454
0.055213058018125594
0.046853877953253686
0.05009761895053089
0.04944572597742081
0.048915919032879174
0.05024164100177586
0.05580371804535389
0.048887974000535905
0.04821258201263845
0.04373652895446867
0.04995268594939262
0.05123829504009336
0.042778225033544004
0.05477324209641665
0.048941295011900365
0.05035764095373452
0.05888436594977975
0.05207320395857096
0.05149410990998149
0.053041924024000764
0.05705916602164507
0.04678505507763475
0.04420923301950097
0.04851782601326704
0.045454569975845516
0.05034

KeyboardInterrupt: 

In [23]:
end_time

63.81341938197147

In [14]:
start_time = timeit.default_timer()
batch = 1
for x,y in cast_train_stream.get_epoch_iterator():
    print("batch {}".format(batch))
    batch = batch +1 

end_time = timeit.default_timer() - start_time

batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82
batch 83
batch 84
batch 85
batch 86
batch 87
batch 88
batch 89
batch 90
batch 91
batch 92
batch 93
batch 94
batch 95
batch 96
batch 97
batch 98
batch 99
batch 100
batch 101
batch 102
batch 103
batch 104
batch 105
batch 106
batch 107
batch 108
batch 109
batch 110
batch 11

In [15]:
end_time

62.48226892598905

In [5]:
iterator= train_stream.get_epoch_iterator()
cur = next(iterator)
cur[0][1].dtype

dtype('float64')

In [ ]:
start_server(train_stream)



In [ ]:
start_time = timeit.default_timer()
x,y = next(iterator)
total_time = timeit.default_timer()-start_time